In [0]:
dbutils.widgets.text("p_data_source","")
v_data_source = dbutils.widgets.get("p_data_source")

In [0]:
%run "/Formula1/includes/configuration"

Out[2]: [FileInfo(path='dbfs:/mnt/formularacedata/presentation/', name='presentation/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/formularacedata/processed/', name='processed/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/formularacedata/raw/', name='raw/', size=0, modificationTime=0)]

In [0]:
%run "/Formula1/includes/common_functions"

whenever you have a multiline json file make sure to add .option('multiline', True) property.

In [0]:
from pyspark.sql.functions import col

file_path = f'{raw_folder_path}/pit_stops.json'
schema = "raceId int, driverId int, stop String, lap int, time String, duration String, milliseconds int"
df = spark.read.format('json').schema(schema).option('multiLine',True).load(file_path)
df.show(3)


+------+--------+----+---+--------+--------+------------+
|raceId|driverId|stop|lap|    time|duration|milliseconds|
+------+--------+----+---+--------+--------+------------+
|   841|     153|   1|  1|17:05:23|  26.898|       26898|
|   841|      30|   1|  1|17:05:52|  25.021|       25021|
|   841|      17|   1| 11|17:20:48|  23.426|       23426|
+------+--------+----+---+--------+--------+------------+
only showing top 3 rows



In [0]:
df.printSchema()

root
 |-- raceId: integer (nullable = true)
 |-- driverId: integer (nullable = true)
 |-- stop: string (nullable = true)
 |-- lap: integer (nullable = true)
 |-- time: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- milliseconds: integer (nullable = true)



In [0]:
from pyspark.sql.functions import current_timestamp
new_df = df.withColumn('ingestion_date', current_timestamp())\
            .withColumnRenamed("race_id","race_id") \
            .withColumnRenamed("driverId", "driver_id")

new_df.show(4)

+------+---------+----+---+--------+--------+------------+--------------------+
|raceId|driver_id|stop|lap|    time|duration|milliseconds|      ingestion_date|
+------+---------+----+---+--------+--------+------------+--------------------+
|   841|      153|   1|  1|17:05:23|  26.898|       26898|2023-06-29 19:45:...|
|   841|       30|   1|  1|17:05:52|  25.021|       25021|2023-06-29 19:45:...|
|   841|       17|   1| 11|17:20:48|  23.426|       23426|2023-06-29 19:45:...|
|   841|        4|   1| 12|17:22:34|  23.251|       23251|2023-06-29 19:45:...|
+------+---------+----+---+--------+--------+------------+--------------------+
only showing top 4 rows



In [0]:
dbutils.fs.ls(processed_folder_path)

Out[11]: [FileInfo(path='dbfs:/mnt/formularacedata/processed/circuits/', name='circuits/', size=0, modificationTime=1686657879000),
 FileInfo(path='dbfs:/mnt/formularacedata/processed/constructors/', name='constructors/', size=0, modificationTime=1686899550000),
 FileInfo(path='dbfs:/mnt/formularacedata/processed/driver/', name='driver/', size=0, modificationTime=1686917781000),
 FileInfo(path='dbfs:/mnt/formularacedata/processed/drivers/', name='drivers/', size=0, modificationTime=1688066523000),
 FileInfo(path='dbfs:/mnt/formularacedata/processed/lap_time/', name='lap_time/', size=0, modificationTime=1686942708000),
 FileInfo(path='dbfs:/mnt/formularacedata/processed/pit_stops/', name='pit_stops/', size=0, modificationTime=1686925132000),
 FileInfo(path='dbfs:/mnt/formularacedata/processed/qualifying/', name='qualifying/', size=0, modificationTime=1686942774000),
 FileInfo(path='dbfs:/mnt/formularacedata/processed/races/', name='races/', size=0, modificationTime=1686741009000),
 File

In [0]:
final_df =  new_df.withColumnRenamed('raceID','race_id')
final_df.show(4)

+-------+---------+----+---+--------+--------+------------+--------------------+
|race_id|driver_id|stop|lap|    time|duration|milliseconds|      ingestion_date|
+-------+---------+----+---+--------+--------+------------+--------------------+
|    841|      153|   1|  1|17:05:23|  26.898|       26898|2023-06-29 19:47:...|
|    841|       30|   1|  1|17:05:52|  25.021|       25021|2023-06-29 19:47:...|
|    841|       17|   1| 11|17:20:48|  23.426|       23426|2023-06-29 19:47:...|
|    841|        4|   1| 12|17:22:34|  23.251|       23251|2023-06-29 19:47:...|
+-------+---------+----+---+--------+--------+------------+--------------------+
only showing top 4 rows



In [0]:
merge_condition = "tgt.race_id = src.race_id AND tgt.driver_id = src.driver_id AND tgt.stop = src.stop AND tgt.race_id = src.race_id"
merge_delta_data(final_df, 'f1_processed', 'pit_stops', processed_folder_path, merge_condition, 'race_id')

In [0]:
%sql
SELECT * FROM f1_processed.pit_stops LIMIT 5

race_id,driver_id,stop,lap,time,duration,milliseconds,ingestion_date
841,153,1,1,17:05:23,26.898,26898,2023-06-29T19:47:31.172+0000
841,30,1,1,17:05:52,25.021,25021,2023-06-29T19:47:31.172+0000
841,17,1,11,17:20:48,23.426,23426,2023-06-29T19:47:31.172+0000
841,4,1,12,17:22:34,23.251,23251,2023-06-29T19:47:31.172+0000
841,13,1,13,17:24:10,23.842,23842,2023-06-29T19:47:31.172+0000


In [0]:
dbutils.notebook.exit("success")